# Modelo Reto

---


## Instrucciones

<Descripción del Problema>


## Solución


### Importar librerías

Se importan las librerías necesarias para el entorno y el agente.


In [124]:
# Librerías para el agente y modelo
import agentpy as ap
import numpy as np

# Librearías de Visualización y otros
import matplotlib.pyplot as plt

plt.style.use("ggplot")


### Definición de las clases

Se define la clase agente `TrafficLight`, la cuál se encarga de simular a los semáforos, y la clase de modelo `TrafficLightModel`, la cuál se encarga de simular el entorno.


In [125]:
class TrafficLight(ap.Agent):
    def setup(self):
        self.state = 0
        self.states = ["red", "green", "yellow"]
        self.updateState()
        self.time = 0
        self.time_limits = self.p.traffic_times

    def act(self):
        self.time += 1
        if self.time == self.time_limits[self.state]:
            self.time = 0
            self.state = (self.state + 1) % 3
            self.updateState()

    def updateState(self):
        self.current_state = self.states[self.state]


In [126]:
class Car(ap.Agent):
    def setup(self):
        self.speed = 0
        self.max_speed = 10
        self.direction = 0
        self.objective = 0
        self.rotation = 0
        self.x = -500
        self.z = -500

    def act(self):
        if (
            self.model.trafficAgents[0].current_state == "green"
            and self.speed < self.max_speed
        ):
            self.speed += 1

        # Update Position
        if self.direction == 0:
            self.x += self.speed
        elif self.direction == 1:
            self.z += self.speed
        elif self.direction == 2:
            self.z -= self.speed
        elif self.direction == 3:
            self.x -= self.speed

        #! TODO: Just slow down or stop if close to a traffic light

        # If yellow light, slow down
        if self.model.trafficAgents[0].current_state == "yellow":
            self.speed -= 1
            if self.speed < 0:
                self.speed = 0

        # If red light, stop
        if self.model.trafficAgents[0].current_state == "red":
            self.speed = 0

        #! TODO: Slow down if close to another car

        #! TODO: Update direction if in marked area


In [127]:
class StreetModel(ap.Model):
    def setup(self):
        self.trafficAgents = ap.AgentList(self, self.p.traffic_lights, TrafficLight)
        self.trafficAgents.state = ap.AttrIter([1, 1, 0, 0])
        self.trafficAgents.updateState()

        self.carAgents = ap.AgentList(self, self.p.cars, Car)

    def step(self):
        self.trafficAgents.act()
        self.carAgents.act()

    def update(self):
        states = np.array(self.trafficAgents.state)
        stateNames = np.array(self.trafficAgents.current_state)

        carPositions = np.array([self.carAgents.x, self.carAgents.z])
        carSpeeds = np.array([self.carAgents.speed])

        self.record("Traffic Light State", states)
        self.record("Traffic Light State Names", stateNames)
        self.record("Car Positions", carPositions)
        self.record("Car Speeds", carSpeeds)

    def end(self):
        pass


### Simulación

Se definen los parámetros, se crea un objeto de la clase `TrafficLightModel`, y se ejecuta la simulación.


In [128]:
parameters = {
    "traffic_lights": 4,  # Número de semáforos
    "traffic_times": [10, 7, 3],  # Tiempos de cada estado (red, green, yellow)
    "cars": 1,  # Número de autos
    "steps": 100,  # Número de pasos
}


In [129]:
model = StreetModel(parameters)
results = model.run()


Completed: 100 steps
Run time: 0:00:00.055789
Simulation finished


In [130]:
results.variables.StreetModel.head(20)


,Traffic Light State,Traffic Light State Names,Car Positions,Car Speeds
t,,,,
0,"[1, 1, 0, 0]","[green, green, red, red]","[[-500], [-500]]",[[0]]
1,"[1, 1, 0, 0]","[green, green, red, red]","[[-499], [-500]]",[[1]]
2,"[1, 1, 0, 0]","[green, green, red, red]","[[-497], [-500]]",[[2]]
3,"[1, 1, 0, 0]","[green, green, red, red]","[[-494], [-500]]",[[3]]
4,"[1, 1, 0, 0]","[green, green, red, red]","[[-490], [-500]]",[[4]]
5,"[1, 1, 0, 0]","[green, green, red, red]","[[-485], [-500]]",[[5]]
6,"[1, 1, 0, 0]","[green, green, red, red]","[[-479], [-500]]",[[6]]
7,"[2, 2, 0, 0]","[yellow, yellow, red, red]","[[-473], [-500]]",[[5]]
8,"[2, 2, 0, 0]","[yellow, yellow, red, red]","[[-468], [-500]]",[[4]]
